# Utils

In [ ]:
%%capture
import time
start = time.time()
!pip uninstall fsspec -qq -y
!pip install --no-index --find-links ../input/hf-datasets/wheels datasets -qq
!pip install -U --no-build-isolation --no-deps ../input/transformers-master/ -qq

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
torch.set_grad_enabled(False)
import transformers
from transformers import AutoTokenizer, AutoModelForQuestionAnswering,AutoConfig, AutoModel, XLMRobertaTokenizerFast
from datasets import Dataset
from chaii_utils import prepare_validation_features, postprocess_qa_predictions
from tqdm.notebook import tqdm
from sklearn import preprocessing
import torch.nn.functional as F
# from torch.optim.swa_utils import AveragedModel
import collections
import re
import os
from glob import glob
# pretrained_paths = ['../input/rembert-pt']
# pretrained_paths = ['../input/infoxlm-large-squad2']
pretrained_paths = ['../input/rembert-pt', '../input/muril-large-pt/muril-large-cased', '../input/infoxlm-large-squad2', '../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2']

# model_paths = ['../input/0829-rembert']
# model_paths = ['../input/xlm-roberta-4']
model_paths = ['../input/0829-rembert', '../input/muril-ep1-full', '../input/xlm-roberta-4', '../input/rob-lr1e-5-wd0-do01-ds05']
model_exts = [[], [2], [], [3,4]]
mymodels = [False,False,False,False]
max_lengths = [512,512,512,512]
doc_strides = [128,128,128,128]
weights = [1,1,1,1]

# pretrained_paths = ['../input/muril-large-pt/muril-large-cased']
# model_paths = ['../input/muril-ep1-full']
# model_exts = [[0,1,2,3]]
# mymodels = [False]
# max_lengths = [512]
# doc_strides = [128]
# weights = [1]

device = 'cuda'
dosoftmax = False

In [ ]:
def my_mean(list_of_lists):
    maxlen = max([len(l) for l in list_of_lists])
    for i in range(len(list_of_lists)):
        while len(list_of_lists[i]) < maxlen:
            list_of_lists[i].append(np.nan)
    return np.nanmax(list_of_lists, axis=1)

def get_char_logits(examples, features, raw_predictions):
    all_start_logits, all_end_logits = raw_predictions
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    all_char_start_logits = {}
    all_char_end_logits = {}
    for example_index, example in enumerate(tqdm(examples)):
        feature_indices = features_per_example[example_index]
        context = example["context"]
        char_start_logits = [[] for _ in range(len(context))]
        char_end_logits = [[] for _ in range(len(context))]
        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            offset_mapping = features[feature_index]["offset_mapping"]
            for token_index, om in enumerate(offset_mapping):
                if om is None:
                    continue
                start_char_idx, end_char_idx = om[0], om[1]
                for char_index in range(start_char_idx, end_char_idx):
                    char_start_logits[char_index].append(start_logits[token_index])
                    char_end_logits[char_index].append(end_logits[token_index])
        char_start_logits = my_mean(char_start_logits)
        char_start_logits[np.isnan(char_start_logits)] = 0
        char_end_logits = my_mean(char_end_logits)
        char_end_logits[np.isnan(char_end_logits)] = 0
        all_char_start_logits[example['id']] = char_start_logits
        all_char_end_logits[example['id']] = char_end_logits
    
    return all_char_start_logits, all_char_end_logits

def beam_search(char_start_logits, char_end_logits, beam_size=20, max_len=100):
    beam_start_indices = np.argsort(char_start_logits)[-beam_size:]
    beam_end_indices = np.argsort(char_end_logits)[-beam_size:]
    idx_pairs = []
    scores = []
    for sidx in beam_start_indices:
        for eidx in beam_end_indices:
            leng = eidx - sidx
            if leng > 0 and leng < max_len:
                idx_pairs.append([sidx, eidx])
                scores.append(char_start_logits[sidx]+char_end_logits[eidx])
    best_score = -1e5
    for i, score in enumerate(scores):
        if score > best_score:
            best_score = score
            besti = i
    if best_score == -1e5:
        return [0, 0]
    return idx_pairs[besti]

def remove_consdup(a, reserve_head=True):
    array = a if reserve_head else np.flip(a)
    last_ele = 1e5
    for i in range(len(array)):
        if array[i] != last_ele:
            last_ele = array[i]
        else:
            array[i] = -1e5
    return array if reserve_head else np.flip(array)

def get_predictions(examples, all_char_start_logits, all_char_end_logits):
    predictions = {}
    for example_index, example in enumerate(tqdm(examples)):
        id_ = example['id']
        context = example["context"]
        char_start_logits = remove_consdup(all_char_start_logits[id_], reserve_head=True)
        char_end_logits = remove_consdup(all_char_end_logits[id_], reserve_head=False)
        best_start_idx, best_end_idx = beam_search(char_start_logits, char_end_logits, max_len=100)
        predictions[id_] = context[best_start_idx:best_end_idx+1]
    return predictions

def left_strip(s):
    reg = [' ', '\n', '(', ')', '.', ',', "'", '\t', '''"''', '[', ']', '-', '_', '!', '?', '#', '*']
    while True:
        if (s[0] not in reg) or (len(s) < 2):
            break
        s = s[1:]
    return s

def right_strip(s):
    reg = [' ', '\n', '(', ')', '.', ',', "'", '\t', '''"''', '[', ']', '-', '_', '!', '?', '#', '*']
    while True:
        if (s[-1] not in reg) or (len(s) < 2):
            break
        s = s[:-1]
    return s

def find_year(s):
    match = re.match(r'.*([1-3][0-9]{3})', s)
    if match is None:
        return s
    y = match.group(1)
    if y + ' ई.' in s:
        return y + ' ई.'
    if y + ' ई.पू.' in s:
        return y + ' ई.पू.'
    return y

def find_year_v2(s):
    words = s.split()
    year_words = []
    for w in words:
        if len(w) == 4 and w.isdigit():
            year_words.append(w)
    if year_words:
        s1 = ' '.join(year_words)
        if s1 + ' ई.' in s:
            return s1 + ' ई.'
        if s1 + ' ई.पू.' in s:
            return s1 + ' ई.पू.'
        return s1
    return s

def get_number(s):
    s1 = [c for c in s if c.isdigit() or c in [' ', ',']]
    s2 = ''
    for c in s1:
        if s2 + c in s:
            s2 += c
        else:
            break
    if not s2:
        return s
    return s2

def get_country(s):
    tamil_countries = ['ஆப்கானிஸ்தான்', 'அல்பேனியா', 'அல்ஜீரியா', 'அமெரிக்கன் சமோவா',
       'அன்டோரா', 'அங்கோலா', 'அங்குயில்லா', 'ஆண்டிகுவா & பார்புடா',
       'அர்ஜென்டினா', 'ஆர்மேனியா', 'அருபா', 'ஆஸ்திரேலியா', 'ஆஸ்திரியா',
       'அஜர்பைஜான்', 'பஹாமாஸ், தி', 'பஹ்ரைன்', 'பங்களாதேஷ்',
       'பார்படாஸ்', 'பெலாரஸ்', 'பெல்ஜியம்', 'பெலிஸ்', 'பெனின்',
       'பெர்முடா', 'பூடான்', 'பொலிவியா', 'போஸ்னியா & ஹெர்சகோவினா',
       'போட்ஸ்வானா', 'பிரேசில்', 'பிரிட்டிஷ் விர்ஜின் இஸ். ', 'புருனே',
       'பல்கேரியா', 'புர்கினா பாசோ', 'பர்மா', 'புருண்டி', 'கம்போடியா',
       'கேமரூன்', 'கனடா', 'கேப் வெர்டே', 'கேமன் தீவுகள்',
       'மத்திய ஆப்பிரிக்க பிரதிநிதி', 'சாட்', 'சிலி', 'சீனா', 'கொலம்பியா',
       'கொமரோஸ்', 'காங்கோ, டெம். பிரதிநிதி. ', 'காங்கோ, குடியரசு. இன் ',
       'குக் தீவுகள்', 'கோஸ்டா ரிகா', "கோட் டி ஐவரி", 'குரோஷியா',
       'கியூபா', 'சைப்ரஸ்', 'செக் குடியரசு', 'டென்மார்க்', 'ஜிபூட்டி',
       'டொமினிகா', 'டொமினிகன் குடியரசு', 'கிழக்கு திமோர்', 'ஈக்வடார்',
       'எகிப்து', 'எல் சால்வடார்', 'எக்குவடோரியல் கினியா', 'எரித்ரியா',
       'எஸ்டோனியா', 'எத்தியோப்பியா', 'ஃபாரோ தீவுகள்', 'பிஜி', 'பின்லாந்து',
       'பிரான்ஸ்', 'பிரெஞ்சு கயானா', 'பிரெஞ்சு பாலினேசியா', 'கபோன்',
       'காம்பியா, தி', 'காசா பகுதி', 'ஜார்ஜியா', 'ஜெர்மனி', 'கானா',
       'ஜிப்ரால்டர்', 'கிரீஸ்', 'கிரீன்லாந்து', 'கிரெனடா', 'குவாடலூப்',
       'குவாம்', 'குவாத்தமாலா', 'குர்ன்சி', 'கினியா', 'கினியா-பிசாவ்',
       'கயானா', 'ஹைட்டி', 'ஹோண்டுராஸ்', 'ஹாங்காங்', 'ஹங்கேரி',
       'ஐஸ்லாந்து', 'இந்தியா', 'இந்தோனேசியா', 'ஈரான்', 'ஈராக்', 'அயர்லாந்து',
       'ஐல் ஆஃப் மேன்', 'இஸ்ரேல்', 'இத்தாலி', 'ஜமைக்கா', 'ஜப்பான்',
       'ஜெர்சி', 'ஜோர்டான்', 'கஜகஸ்தான்', 'கென்யா', 'கிரிபதி',
       'கொரியா, வடக்கு', 'கொரியா, தெற்கு', 'குவைத்', 'கிர்கிஸ்தான்',
       'லாவோஸ்', 'லாட்வியா', 'லெபனான்', 'லெசோதோ', 'லைபீரியா', 'லிபியா',
       'லிச்டென்ஸ்டீன்', 'லிதுவேனியா', 'லக்சம்பர்க்', 'மக்காவ்',
       'மசிடோனியா', 'மடகாஸ்கர்', 'மலாவி', 'மலேசியா', 'மாலத்தீவுகள்',
       'மாலி', 'மால்டா', 'மார்ஷல் தீவுகள்', 'மார்டினிக்',
       'மௌரிடானியா', 'மொரிஷியஸ்', 'மயோட்', 'மெக்சிகோ',
       'மைக்ரோனேசியா, ஃபெட். செயின்ட்', 'மால்டோவா', 'மொனாக்கோ', 'மங்கோலியா',
       'மான்செராட்', 'மொராக்கோ', 'மொசாம்பிக்', 'நமீபியா', 'நவ்ரு',
       'நேபாளம்', 'நெதர்லாந்து', 'நெதர்லாந்து அண்டிலிஸ்',
       'நியூ கலிடோனியா', 'நியூசிலாந்து', 'நிகரகுவா', 'நைஜர்',
       'நைஜீரியா', 'என். மரியானா தீவுகள்', 'நோர்வே', 'ஓமன்', 'பாகிஸ்தான்',
       'பலாவ்', 'பனாமா', 'பப்புவா நியூ கினியா', 'பராகுவே', 'பெரு',
       'பிலிப்பைன்ஸ்', 'போலந்து', 'போர்ச்சுகல்', 'புவேர்ட்டோ ரிக்கோ', 'கத்தார்',
       'ரீயூனியன்', 'ருமேனியா', 'ரஷ்யா', 'ருவாண்டா', 'செயின்ட் ஹெலினா',
       'செயின்ட் கிட்ஸ் & நெவிஸ்', 'செயின்ட் லூசியா', 'செயின்ட் பியர் & மிக்குலோன்',
       'செயின்ட் வின்சென்ட் மற்றும் கிரெனடைன்ஸ்', 'சமோவா', 'சான் மரினோ',
       'Sao Tome & Principe', 'சவூதி அரேபியா', 'செனகல்', 'செர்பியா',
       'சீஷெல்ஸ்', 'சியரா லியோன்', 'சிங்கப்பூர்', 'ஸ்லோவாக்கியா',
       'ஸ்லோவேனியா', 'சாலமன் தீவுகள்', 'சோமாலியா', 'தென் ஆப்பிரிக்கா',
       'ஸ்பெயின்', 'இலங்கை', 'சூடான்', 'சுரினாம்', 'ஸ்வாசிலாந்து',
       'ஸ்வீடன்', 'சுவிட்சர்லாந்து', 'சிரியா', 'தைவான்', 'தஜிகிஸ்தான்',
       'தான்சானியா', 'தாய்லாந்து', 'டோகோ', 'டோங்கா', 'டிரினிடாட் & டொபாகோ',
       'துனிசியா', 'துருக்கி', 'துர்க்மெனிஸ்தான்', 'டர்க்ஸ் & கெய்கோஸ்',
       'துவாலு', 'உகாண்டா', 'உக்ரைன்', 'ஐக்கிய அரபு எமிரேட்ஸ்',
       'யுனைடெட் கிங்டம்', 'யுனைடெட் ஸ்டேட்ஸ்', 'உருகுவே', 'உஸ்பெகிஸ்தான்',
       'வனுவாடு', 'வெனிசுலா', 'வியட்நாம்', 'விர்ஜின் தீவுகள்',
       'வாலிஸ் அண்ட் ஃபுடுனா', 'வெஸ்ட் பேங்க்', 'வெஸ்டர்ன் சஹாரா', 'யேமன்',
       'சாம்பியா', 'ஜிம்பாப்வே']
    for c in tamil_countries:
        if c in s:
            return c
    return s

def my_pp(s, question, context):
    s = left_strip(s)
    
    # A.G. 
    sp = s.split()
    if len(sp) >= 2 and sp[-1][-1] == '.' and sp[-2][-1] == '.':
        return s
    
#     # xxx (xxx
#     if '(' in s:
#         s = s[:s.find('(')]
    
    s = right_strip(s)
    
    
    # nico pp
    tamil_ad = "கி.பி"
    tamil_bc = "கி.மு"
    tamil_km = "கி.மீ"
    hindi_ad = "ई"
    hindi_bc = "ई.पू"
    if any([s.endswith(tamil_ad), s.endswith(tamil_bc), s.endswith(tamil_km), s.endswith(hindi_ad), s.endswith(hindi_bc)]) and s+"." in context:
        s = s+"."
    
#     # hindi which year + tamil which year எந்த ஆண்டு
    if 'किस वर्ष' in question or 'எந்த ஆண்டு' in question:
        s = find_year(s)
    
    # hindi which year v2
#     if 'किस वर्ष' in question:
#         s = find_year_v2(s)
    
    # tamil area
    if question.endswith('பரப்பளவு என்ன?'):
        if s[-1].isnumeric():
            if s + ' சதுர கிலோமீட்டர்கள்' in context:
                s = s + ' சதுர கிலோமீட்டர்கள்'
            elif s + ' சதுர கிலோ மீட்டர்' in context:
                s = s + ' சதுர கிலோ மீட்டர்'
            elif s + ' சதுர கிலோ மீட்டர்கள்' in context:
                s = s + ' சதுர கிலோ மீட்டர்கள்'
            elif s + ' சதுர மைல்கள்' in context:
                s = s + ' சதுர மைல்கள்'
            elif s + ' சதுர' in context:
                s = s + ' சதுர'
            elif s + ' கி.மீ²' in context:
                s = s + ' கி.மீ²'
        else:
            if s + '²' in context:
                s = s + '²'
            elif s + '2' in context:
                s = s + '2'
    
    # hindi number
    if 'संख्या' in question:
        s = get_number(s)
        
    #a(b) -> a
    if '(' in s:
        s = s[:s.find('(')]
        
    #tamil country extract
    if question.endswith('நாடு எது?') or question.startswith('எந்த நாடு'):
        s = get_country(s)
    
    # tamil years old
    if 'வயதில்' in question:
        s = get_number(s)
        
    # negative number recover
#     if s[0].isdigit():
#         if context.count(s) == context.count(' -' + s):
#             s = '-' + s        
        
    # fix typos
    # japan
    if s == 'சப்பான்' and 'ஜப்பான்' in context:
        return 'ஜப்பான்'
    # mubai
    if s == 'मुंबई' and 'मुम्बई' in context:
        return 'मुम्बई'
        
    return s
    
        
def softmax(array):
    ctx = np.exp(array)
    return ctx / np.sum(ctx)

def tez_pp(s):
    from string import punctuation
    s = " ".join(s.split())
    s = s.strip(punctuation)
    return s

def get_len(x):
    x['seq_len'] = len(x['attention_mask'])
    return x

def get_batches(features, batch_size=16, pad_id=0):
    attention_mask = features['attention_mask']
    input_ids = features['input_ids']
    ret = []
    for i in tqdm(range(0, len(attention_mask), batch_size)):
        batch_attention_mask = attention_mask[i:i+batch_size]
        batch_input_ids = input_ids[i:i+batch_size]
        maxlen = max([len(x) for x in batch_attention_mask])
        padded_batch_attention_mask = [x+[pad_id]*(maxlen-len(x)) for x in batch_attention_mask]
        padded_batch_input_ids = [x+[pad_id]*(maxlen-len(x)) for x in batch_input_ids]
        batch = {
            'attention_mask': torch.tensor(padded_batch_attention_mask, device=device),
            'input_ids': torch.tensor(padded_batch_input_ids, device=device),
        }
        ret.append(batch)
    return ret

def pad_back(logits):
    return F.pad(logits, pad=(0, 512-logits.shape[1]), mode='constant', value=-1e5)

# Infer

In [ ]:
test = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/test.csv')
test_dataset = Dataset.from_pandas(test)
example_id_to_index = {k: i for i, k in enumerate(test_dataset["id"])}
blend_start_logits, blend_end_logits = collections.defaultdict(list), collections.defaultdict(list)
for i in range(len(pretrained_paths)):
    print(f'#model: {i}')
    start1 = time.time()
    pretrained_path = pretrained_paths[i]
    model_path = model_paths[i]
    mymodel = mymodels[i]
    max_length = max_lengths[i]
    doc_stride = doc_strides[i]
    
    if i != 3:
        tokenizer = AutoTokenizer.from_pretrained(pretrained_path)
        pad_on_right = tokenizer.padding_side == "right"
        test_features = test_dataset.map(
            lambda x: prepare_validation_features(x, tokenizer, max_length, doc_stride, pad_on_right, padding=False),
            batched=True,
            remove_columns=test_dataset.column_names,
            num_proc=2
        )
        test_features = test_features.map(get_len, num_proc=2)
        test_features = test_features.sort('seq_len')
        test_dataloader = get_batches(test_features, batch_size=16, pad_id=tokenizer.pad_token_id)
    model = ChaiiModelLoadHead(pretrained_path).to(device) if mymodel else AutoModelForQuestionAnswering.from_pretrained(pretrained_path).to(device)
    model.eval()
    all_start_logits = []
    all_end_logits = []
    print(f'init time {(time.time()-start1)//60} minutes')
    
    start1 = time.time()
    exts = model_exts[i]
    weight_paths = glob(os.path.join(model_path, '*.pt'))
    for ii, ext in enumerate(exts):
        weight_paths = [x for x in weight_paths if f'{ext}.pt' not in x]
    for weight_path in weight_paths:
        print('weight:', weight_path)
        model.load_state_dict(torch.load(weight_path))
        start_logits = []
        end_logits = []
        for batch in tqdm(test_dataloader, leave=False):
            pred = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
            start_logits.append(pad_back(pred.start_logits))
            end_logits.append(pad_back(pred.end_logits))
        start_logits = torch.cat(start_logits, dim=0)
        end_logits = torch.cat(end_logits, dim=0)
        start_logits = start_logits.cpu().numpy()
        end_logits = end_logits.cpu().numpy()
        all_start_logits.append(start_logits)
        all_end_logits.append(end_logits)
    all_start_logits = np.array(all_start_logits)
    all_end_logits = np.array(all_end_logits)
    all_start_logits = np.mean(all_start_logits, axis=0)
    all_end_logits = np.mean(all_end_logits, axis=0)
    raw_predictions = [all_start_logits, all_end_logits]
    print(f'inference time {(time.time()-start1)//60} minutes')
    
    start1 = time.time()
    all_char_start_logits, all_char_end_logits = get_char_logits(test_dataset, test_features, raw_predictions)
    for k in all_char_start_logits:
        if dosoftmax:
            blend_start_logits[k].append(softmax(all_char_start_logits[k]))
            blend_end_logits[k].append(softmax(all_char_end_logits[k]))
        else:
            blend_start_logits[k].append(all_char_start_logits[k])
            blend_end_logits[k].append(all_char_end_logits[k])
    print(f'char mapping time {(time.time()-start1)//60} minutes')

# Blend

In [ ]:

blended_start_logits, blended_end_logits = {}, {}
for k in blend_start_logits:
    blended_start_logits[k] = np.average(np.array(blend_start_logits[k]), axis=0, weights=weights)
    blended_end_logits[k] = np.average(np.array(blend_end_logits[k]), axis=0, weights=weights)
predictions = get_predictions(test_dataset, blended_start_logits, blended_end_logits)
predictions

In [ ]:
sub = test.copy().reset_index(drop=True)
sub['PredictionString'] = sub['id'].apply(lambda r: predictions[r])
# sub['PredictionString'] = sub['PredictionString'].apply(my_pp)
# sub['PredictionString'] = sub['PredictionString'].apply(tez_pp)
for i in range(len(sub)):
    pred = sub.loc[i, 'PredictionString']
    ctx = sub.loc[i, 'context']
    qst = sub.loc[i, 'question']
    sub.loc[i, 'PredictionString'] = my_pp(pred, qst, ctx)


In [ ]:
train = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/train.csv')

In [ ]:
# test for leak
for i in range(len(sub)):
    question = sub.loc[i, 'question']
    context = sub.loc[i, 'context']
    matches = train[train['question'] == question].reset_index(drop=True)
    for j in range(len(matches)):
        new_ans = matches.loc[j, 'answer_text']
        if new_ans in context:
            sub.loc[i, 'PredictionString'] = new_ans
            break
    

In [ ]:
sub = sub[['id', 'PredictionString']]
sub.to_csv('submission.csv', index=False)
sub

In [ ]:
duration = time.time() - start
print(f'duration: {duration//60} minutes')